# اتصال به گوکل کولب

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# خواندن اطلاعات

In [ ]:
import pandas as pd
df = pd.read_excel (r'/content/drive/My Drive/DataSet/Community Detection/Sim.xlsx')
Data = df.values.tolist()

3 Group

In [ ]:
import pandas as pd
Target3=[]
df = pd.read_excel (r'/content/drive/My Drive/DataSet/Community Detection/3Group.xlsx')
Gropup_3 = df.values.tolist()
for i in range(0,len(Gropup_3)):
 Target3.append(Gropup_3[i][0]-1)

5 Group

In [ ]:
import pandas as pd
Target5=[]
df = pd.read_excel (r'/content/drive/My Drive/DataSet/Community Detection/5Group.xlsx')
Gropup_5 = df.values.tolist()
for i in range(0,len(Gropup_5)):
 Target5.append(Gropup_5[i][0]-1)

10 Group

In [ ]:
import pandas as pd
Target10=[]
df = pd.read_excel (r'/content/drive/My Drive/DataSet/Community Detection/10Group.xlsx')
Gropup_10 = df.values.tolist()
for i in range(0,len(Gropup_10)):
 Target10.append(Gropup_10[i][0]-1)

15 Group

In [ ]:
import pandas as pd
Target15=[]
df = pd.read_excel (r'/content/drive/My Drive/DataSet/Community Detection/15Group.xlsx')
Gropup_15 = df.values.tolist()
for i in range(0,len(Gropup_15)):
 Target15.append(Gropup_15[i][0]-1)

# تقسیم بندی داده ها

In [ ]:
k=3   # For Group Selection  3,5,10,15

In [ ]:
Class=0 # intial 
if(k==3):
  Target=Target3
  Class=3
elif(k==5):
  Target=Target5
  Class=5
elif(k==10):
  Target=Target10
  Class=10
else:
  Target=Target15
  Class=15

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data, Target, test_size=0.3, random_state=42)

# پیش پردازش

In [ ]:
from tensorflow.keras.utils import to_categorical
trainy = to_categorical(y_train)
testy = to_categorical(y_test)

In [ ]:
import numpy as np
trainX=[]
for i in range(0,len(X_train)):
  x=np.array(X_train[i])
  trainX.append(np.tile(x,(10,1)))

In [ ]:
testX=[]
for i in range(0,len(X_test)):
  x=np.array(X_test[i])
  testX.append(np.tile(x,(10,1)))

In [ ]:
trainX=np.array(trainX)
testX=np.array(testX)
trainX=np.reshape(trainX,(len(trainX),10,943,1))
testX=np.reshape(testX,(len(testX),10,943,1))

# استخراج ویژگی

In [ ]:
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.models import Model

myInput=layers.Input(shape=(10,943,1))
conv1=SeparableConv2D(64,3, activation='relu',padding='same')(myInput)
max1=MaxPooling2D(2)(conv1)
conv2=SeparableConv2D(64,3, activation='relu',padding='same')(max1)
max2=MaxPooling2D(2)(conv2)
flat=Flatten()(max2)
des1 =Dense(50, activation='relu')(flat)
Extract_Model = Model(myInput, des1)
des2 =Dense(10, activation='relu')(des1)
out_layer =Dense(Class, activation='softmax')(des2)
model = Model(myInput, out_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 943, 1)]      0         
                                                                 
 separable_conv2d (Separable  (None, 10, 943, 64)      137       
 Conv2D)                                                         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 471, 64)       0         
 )                                                               
                                                                 
 separable_conv2d_1 (Separab  (None, 5, 471, 64)       4736      
 leConv2D)                                                       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 235, 64)       0         
 2D)                                                       

In [ ]:
verbose, epochs, batch_size = 1, 20, 32
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/20
21/21 [==============================] - 10s 406ms/step - loss: 1.0341 - accuracy: 0.4765
Epoch 2/20
21/21 [==============================] - 9s 411ms/step - loss: 0.5333 - accuracy: 0.8149
Epoch 3/20
21/21 [==============================] - 9s 424ms/step - loss: 0.3507 - accuracy: 0.8589
Epoch 4/20
21/21 [==============================] - 9s 441ms/step - loss: 0.2269 - accuracy: 0.9256
Epoch 5/20
21/21 [==============================] - 10s 463ms/step - loss: 0.1627 - accuracy: 0.9454
Epoch 6/20
21/21 [==============================] - 9s 423ms/step - loss: 0.1227 - accuracy: 0.9605
Epoch 7/20
21/21 [==============================] - 9s 424ms/step - loss: 0.1275 - accuracy: 0.9378
Epoch 8/20
21/21 [==============================] - 9s 428ms/step - loss: 0.0894 - accuracy: 0.9651
Epoch 9/20
21/21 [==============================] - 9s 423ms/step - loss: 0.0753 - accuracy: 0.9788
Epoch 10/20
21/21 [==============================] - 11s 522ms/step - loss: 0.0877 - accuracy: 0.9

# طبقه بندی

In [ ]:
trainFeature=(Extract_Model.predict(trainX))
testFeature=(Extract_Model.predict(testX))

9/9 [==============================] - 1s 110ms/step


In [ ]:
from sklearn import svm
classifier_svm = svm.LinearSVC()
classifier_svm.fit(trainFeature, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [ ]:
from sklearn import tree
classifier_tree = tree.DecisionTreeClassifier()
classifier_tree.fit(trainFeature, y_train)

DecisionTreeClassifier()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_Knn = KNeighborsClassifier(n_neighbors=3)
classifier_Knn.fit(trainFeature, y_train)

KNeighborsClassifier(n_neighbors=3)

# ترکیب طبقه بند

In [ ]:
from sklearn.ensemble import  VotingClassifier
classifier_Voting = VotingClassifier(estimators=[('SVM', classifier_svm), ('TREE', classifier_tree), ('KNN', classifier_Knn)],weights=[2,1,1])
classifier_Voting.fit(trainFeature, y_train)
Class_Voting=classifier_Voting.predict(testFeature)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# ارزیابی

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test,Class_Voting))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98        81
           1       0.99      1.00      1.00       115
           2       1.00      0.94      0.97        87

    accuracy                           0.98       283
   macro avg       0.98      0.98      0.98       283
weighted avg       0.98      0.98      0.98       283



In [ ]:
print('Accuracy= ' + str(round(accuracy_score(y_test, Class_Voting)*100))) 

Accuracy= 98


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, Class_Voting)

array([[ 81,   0,   0],
       [  0, 115,   0],
       [  4,   1,  82]])